In [232]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn

In [5]:
input_size, hidden_size = 3, 2

#### toy rnn to predict next number in arithmetic sequence

In [34]:
class DemoRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, nonlinearity='relu', batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x, h_t = self.rnn.forward(x)
        x = self.linear.forward(x)
        return x
    
rnn = DemoRNN(input_size, 1)

In [35]:
rnn.state_dict()

OrderedDict([('rnn.weight_ih_l0', tensor([[ 0.9426, -0.4727, -0.5023]])),
             ('rnn.weight_hh_l0', tensor([[0.8646]])),
             ('rnn.bias_ih_l0', tensor([0.0567])),
             ('rnn.bias_hh_l0', tensor([0.5446])),
             ('linear.weight', tensor([[-0.1725]])),
             ('linear.bias', tensor([0.0035]))])

In [36]:
input = torch.Tensor([1, 2, 3])
input = input.reshape(1, 1, 3)

In [37]:
input.size() # batch_size x times_steps x features

torch.Size([1, 1, 3])

In [38]:
rnn.forward(input)

tensor([[[0.0035]]], grad_fn=<ViewBackward0>)

#### (fake) data prep

In [204]:
import random

def create_sequence():
    interval = random.randint(1, 50)
    start = random.randint(0, 100)
    seq = np.arange(start, start + interval*3.5, interval)
    return seq

In [230]:
create_sequence()

array([ 56.,  82., 108., 134.])

In [205]:
x, y = [], []

for _ in range(10000):
    seq = create_sequence()
    *x_i, y_i = seq
    x.append(x_i)
    y.append(y_i)
    assert len(x_i) == 3 and (type(y_i) is np.float64)

x = np.array(x)
y = np.array(y)

In [206]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [207]:
print(x_train.shape, y_train.shape)

(8000, 3) (8000, 1)


In [208]:
xy_train = np.concatenate((x_train, y_train), axis=1)

In [209]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xy_train_normalized = torch.tensor(scaler.fit_transform(xy_train)).float()

In [211]:
x_train_normalized = xy_train_normalized[:, 0:3]
y_train_normalized = xy_train_normalized[:, 3:4]

### training

In [212]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(x_train_normalized, y_train_normalized)
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [213]:
seq_rnn = DemoRNN(3, 200)
optimizer = torch.optim.Adam(seq_rnn.parameters())

epoch = 4
for _ in range(epoch):
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()
        y_pred = seq_rnn.forward(x_batch)
        loss = F.mse_loss(y_pred, y_batch)
        loss.backward()
        optimizer.step()

#### eval

In [214]:
def scale_x(x, scaler):
    data_sample = np.concatenate((x, [0])).reshape(1, 4)
    data_sample_scaled = scaler.transform(data_sample)
    return data_sample_scaled[:,:3]

In [215]:
def inverse_scale_y(y, scaler):
    arr = np.array([[0, 0, 0, y]])
    arr = scaler.inverse_transform(arr)
    return arr[0][-1]

In [216]:
def predict_next(seq, model):
    with torch.no_grad():
        ds_tensor = torch.tensor(scale_x(seq, scaler)).float()
        print(ds_tensor)
        y_pred = model.forward(ds_tensor)
        y_unscaled = inverse_scale_y(y_pred.item(), scaler)
        print(y_unscaled)
        

In [231]:
predict_next([100, 150, 200], seq_rnn)

tensor([[1., 1., 1.]])
249.3495814204216
